[Оригинал](https://pbpython.com/pandas_transform.html)

Рассмотрим пример данных из 12 сделок купли-продажи:

In [ ]:
import pandas as pd

df = pd.read_excel("data/sales_transactions.xlsx")
df.head()

In [ ]:
df.order.unique()

Из данных видно, что файл содержит `3` разных заказа (`10001`, `10005` и `10006`) и что каждый заказ состоит из нескольких продуктов (иначе говоря, `sku`).

Вопрос, на который мы хотели бы ответить: «Какой процент от общей суммы заказа представляет каждый `sku`?»

Например, если мы посмотрим на заказ `10001` на общую сумму `576,12` доллара, разбивка будет выглядеть так:

Сложность в этом расчете заключается в том, что нам нужно получить итоговую сумму для каждого заказа и объединить ее с деталями уровня транзакции, чтобы получить проценты. В `Excel` вы можете попытаться использовать некоторую версию промежуточного итога, чтобы попытаться вычислить значения.

### Первый подход - слияние

Если вы знакомы с `pandas`, вашим первым желанием будет попытаться сгруппировать данные в новый фрейм и объединить их в многошаговый процесс.

Теперь, когда данные находятся в датафрейме, можно определить итоговую сумму по `order` с помощью стандартного агрегирования `groupby`.

In [ ]:
df.groupby('order')["ext price"].sum()

Вот простое изображение, показывающее, что происходит:

<img src="https://pbpython.com/images/groupby-example.png">

Более сложная задача - выяснить, как объединить эти данные с исходным фреймом. Первая мысль - создать новый фрейм с итогами по `order` и объединить его с оригиналом. Мы могли бы сделать что-то вроде такого:

In [ ]:
order_total = df.groupby('order')["ext price"].sum().rename("Order_Total").reset_index()
df_1 = df.merge(order_total)
df_1["Percent_of_Order"] = round(df_1["ext price"] / df_1["Order_Total"] * 100)
df_1

### Второй подход - использование преобразования

Используя исходные данные, давайте попробуем использовать `transform` и `groupby`:

In [ ]:
df.groupby('order')["ext price"].transform('sum')

Вы заметите, что возвращается набор данных другого размера из наших обычных `groupby` функций. Вместо того, чтобы показывать только итоги для 3 заказов, мы сохраняем то же количество элементов, что и исходный набор данных. Это уникальная особенность использования `transform`.

Последний шаг довольно прост:

In [ ]:
df["Order_Total"] = df.groupby('order')["ext price"].transform('sum')
df["Percent_of_Order"] = round(df["ext price"] / df["Order_Total"] * 100)
df

В качестве дополнительного бонуса вы можете записать одно выражение, если не хотите показывать отдельные суммы заказа:

In [ ]:
df["Percent_of_Order"] = round(df["ext price"] / 
                               df.groupby('order')["ext price"].transform('sum') * 100)
df

Вот диаграмма, чтобы показать, что происходит:

<img src="https://pbpython.com/images/transform-example.png">